<img src="NotebookAddons/blackboard-banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>Change Detection in <font color='rgba(200,0,0,0.2)'>Your Own</font> SAR Amplitude Time Series Stack </b> </font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, <a href="http://earthbigdata.com/" target="_blank">Earth Big Data, LLC</a> </b> <br>
<img style="padding: 7px" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right"/>
</font>

<font size="3"> This notebook introduces you to the methods of change detection in deep multi-temporal SAR image data stacks.
<br><br>
<b>In this chapter we introduce the following data analysis concepts:</b>

- How to use your own HyP3-generated data stack in a change detection effort
- The concepts of time series slicing by month, year, and date.
- The concepts and workflow of Cumulative Sum-based change point detection.
- The identification of change dates for each identified change point.
</font>
</font>

<hr>
<font face="Calibri" size="5" color="darkred"> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://pandas.pydata.org/" target="_blank">Pandas</a></b> is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. </li>
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
    <li> The <b><a href="https://hyp3-docs.asf.alaska.edu/using/sdk/" target="_blank">HyP3 SDK</a></b> is a python package for requesting and accessing On Demand SAR data generated by the Alaska Satellite Facility's Hybrid Pluggable Processing Pipeline, or HyP3 (pronounced "hype"). </li>
<li><b><a href="https://www.scipy.org/about.html" target="_blank">SciPy</a></b> is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics. </li>

</font>

<font face="Calibri" size="3"><b>Import the necessary modules and libraries:</b></font>

In [ ]:
%%capture
import copy
import os
import glob
import shutil
import re

import pandas as pd
import numpy as np
import pyproj
from osgeo import gdal, osr

%matplotlib notebook
import matplotlib.pylab as plt
from matplotlib import animation
from matplotlib import rc

from IPython.display import HTML
from IPython.display import Markdown
from IPython.display import display
from IPython.display import clear_output

import asf_notebook as asfn
asfn.jupytertheme_matplotlib_format()

try:
    from hyp3_sdk import Batch
except ImportError:
    !python -m pip install hyp3-sdk==0.5 --user
    from hyp3_sdk import Batch

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Your Own Data Stack Into the Notebook </b> </font>

<font size="3"> This lab assumes that you've created your own data stack over your personal area of interest using the Alaska Satellite Facility's <a href="https://search.asf.alaska.edu/#/?topic=onDemand" target="_blank">On Demand RTC processing</a> via <a href="https://hyp3-docs.asf.alaska.edu/" target="_blank">HyP3</a>.

This lab expects <a href="https://hyp3-docs.asf.alaska.edu/guides/rtc_atbd/" target="_blank">Radiometric Terrain Corrected</a> (RTC) image products as input. Prefer a unique orbit geometry **(choose ascending or descending, not both)** to keep geometric differences between images low.

We will retrieve HyP3 data via the HyP3 SDK. As both HyP3 and the Notebook environment sit in the <a href="https://aws.amazon.com/" target="_blank">Amazon Web Services (AWS)</a> cloud, data transfer is quick and cost effective.</font>
</font>

<font face="Calibri" size="4"> <b>1.1 Login to HyP3 and Download Products</b> <br>

<font face="Calibri" size="3"><b>Create a HyP3 object and authenticate with your Earthdata credentials</b> </font>

In [ ]:
hyp3 = asfn.hyp3_auth()

<hr>
<font face="Calibri" size="3"> Before we download anything, create a working directory for this analysis and change into it.
<br><br>
<b>Select or create a working directory for the analysis:</b></font>

In [ ]:
while True:
    sub_dir = asfn.input_path(
        f"\nPlease enter the name of a directory in which to store your data for this analysis.")
    if os.path.exists(sub_dir):
        contents = glob.glob(f'{sub_dir}/*')
        if len(contents) > 0:
            choice = asfn.handle_old_data(sub_dir, contents)
            if choice == 1:
                shutil.rmtree(sub_dir)
                os.mkdir(sub_dir)
                break
            if choice == 2:
                break
            else:
                clear_output()
                continue
        else:
            break
    else:
        os.mkdir(sub_dir)
        break

<font face="Calibri" size="3"><b>Change into the analysis directory:</b></font>

In [ ]:
analysis_directory = f"{os.getcwd()}/{sub_dir}"
os.chdir(analysis_directory)
print(f"Current working directory: {os.getcwd()}")

<font face="Calibri" size="3"><b>Create a folder in which to download your RTC products.</b> </font>

In [ ]:
rtc_path = "rtc_products"
asfn.new_directory(rtc_path)
products_path = f"{analysis_directory}/{rtc_path}"

<font face="Calibri" size="3"><b>List projects and select one:</b></font>

In [ ]:
projects = asfn.get_RTC_projects(hyp3)

if len(projects) > 0:
    display(Markdown("<text style='color:darkred;'>Note: After selecting a project, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:darkred;'>Otherwise, you will simply rerun this code cell.</text>"))
    print('\nSelect a Project:')
    project_select = asfn.select_parameter(projects)

project_select

<font face="Calibri" size="3"><b>Select a date range of products to download:</b> </font>

In [ ]:
project = project_select.value
jobs = hyp3.find_jobs(name=project)
jobs = jobs.filter_jobs(running=False, include_expired=False)
jobs = Batch([job for job in jobs if job.job_type.startswith('RTC')])

if len(jobs) < 1:
    raise ValueError("There are no unexpired RTC products for this project.\nSelect a different project or rerun your jobs in Vertex.")

display(Markdown("<text style='color:darkred;'>Note: After selecting a date range, you should select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you may simply rerun this code cell.</text>"))
print('\nSelect a Date Range:')
dates = asfn.get_job_dates(jobs)
date_picker = asfn.gui_date_picker(dates)
date_picker

<font face="Calibri" size="3"><b>Save the a date range:</b> </font>

In [ ]:
date_range = asfn.get_slider_vals(date_picker)
date_range[0] = date_range[0].date()
date_range[1] = date_range[1].date()
print(f"Date Range: {str(date_range[0])} to {str(date_range[1])}")
project = asfn.filter_jobs_by_date(jobs, date_range)

<font face="Calibri" size="3"><b>Gather the available paths and orbit directions for the remaining products:</b></font>

In [ ]:
display(Markdown("<text style='color:darkred;'><text style='font-size:150%;'>This may take some time for projects containing many jobs...</text></text>"))
project = asfn.get_paths_orbits(project)
paths = set()
orbit_directions = set()
for p in project:
    paths.add(p.path)
    orbit_directions.add(p.orbit_direction)
paths.add('All Paths')
display(Markdown(f"<text style=color:blue><text style='font-size:175%;'>Done.</text></text>"))

<hr>
<font face="Calibri" size="3"><b>Select a path or paths (use shift or ctrl to select multiple paths):</b></font>

In [ ]:
display(Markdown("<text style='color:darkred;'>Note: After selecting a path, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Path:')
path_choice = asfn.select_mult_parameters(paths)
path_choice

<font face="Calibri" size="3"><b>Save the selected flight path/s:</b></font>

In [ ]:
flight_path = path_choice.value
if flight_path:
    if 'All Paths' not in flight_path:
        print(f"Flight Path: {flight_path}")
    else:
        print('Flight Path: All Paths')
else:
    print("WARNING: You must select a flight path in the previous cell, then rerun this cell.")

<font face="Calibri" size="3"><b>Select an orbit direction:</b></font>

In [ ]:
if len(orbit_directions) > 1:
    display(Markdown("<text style='color:red;'>Note: After selecting a flight direction, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:red;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Flight Direction:')
direction_choice = asfn.select_parameter(orbit_directions, 'Direction:')
direction_choice

<font face="Calibri" size="3"><b>Save the selected orbit direction:</b></font>

In [ ]:
direction = direction_choice.value
print(f"Orbit Direction: {direction}")

<font face="Calibri" size="3"><b>Filter jobs by path and orbit direction:</b></font>

In [ ]:
project = asfn.filter_jobs_by_path(project, flight_path)
project = asfn.filter_jobs_by_orbit(project, direction)
print(f"There are {len(project)} products to download.")

<font face="Calibri" size="3"><b>Download the products, unzip them into the rtc_products directory, and delete the zip files:</b> </font>

In [ ]:
print(f"\nProject: {project.jobs[0].name}")
project_zips = project.download_files()
for z in project_zips:
    asfn.asf_unzip(products_path, str(z))
    z.unlink()

<font face="Calibri" size="3"><b>Determine the available polarizations:</b></font>

In [ ]:
polarizations = asfn.get_RTC_polarizations(rtc_path)
polarization_power_set = asfn.get_power_set(polarizations)

<font face="Calibri" size="3"><b>Select a polarization:</b></font>

In [ ]:
polarization_choice = asfn.select_parameter(sorted(polarization_power_set), 'Polarizations:')
polarization_choice

<font face="Calibri" size="3"><b>Create a paths variable, holding the relative path to the tiffs in the selected polarization/s:</b></font>

In [ ]:
polarization = polarization_choice.value
print(polarization)
if len(polarization) == 2:
    regex = "\w[\--~]{{5,300}}(_|-){}.(tif|tiff)$".format(polarization)
    dbl_polar = False
else:
    regex = "\w[\--~]{{5,300}}(_|-){}(v|V|h|H).(tif|tiff)$".format(polarization[0])
    dbl_polar = True

<font face="Calibri" size="3"><b>Write functions to collect and print the paths of the tiffs:</b></font>

In [ ]:
def get_tiff_paths(regex, pths):
    tiff_paths = []
    for pth in glob.glob(pths):
        tiff_path = re.search(regex, pth)
        if tiff_path:
            tiff_paths.append(pth)
    return tiff_paths

def print_tiff_paths(tiff_paths):
    print("Tiff paths:")
    for p in tiff_paths:
        print(f"{p}\n")

<font face="Calibri" size="3"><b>Collect and print the paths of the tiffs:</b></font>

In [ ]:
tiff_pth = f"{rtc_path}/*/*{polarization[0]}*.tif*"
tiff_paths = get_tiff_paths(regex, tiff_pth)
print_tiff_paths(tiff_paths)

<hr>
<font face="Calibri" size="4"> <b>1.2 Fix multiple UTM Zone-related issues</b> <br>
<br>
<font face="Calibri" size="3">Fix multiple UTM Zone-related issues should they exist in your data set. If multiple UTM zones are found, the following code cells will identify the predominant UTM zone and reproject the rest into that zone. This step must be completed prior to merging frames or performing any analysis.</font>
<br><br>
<font face="Calibri" size="3"><b>Select a source data coordinate system:</b>
<br>
Note: You can select Lat/Long if your source data is in UTM, and it will be reprojected into Lat/Long, EPSG 4326. </font>

In [ ]:
coord_choice = asfn.select_parameter(["UTM", "Lat/Long"], 'Coord Systems:')
coord_choice

In [ ]:
utm_zones = []
utm_types = []
print('Checking UTM Zones in the data stack ...\n')
for k in range(0, len(tiff_paths)):
    info = gdal.Info(tiff_paths[k], format='json')
    proj = osr.SpatialReference(info['coordinateSystem']['wkt'])
    utm_zones.append(int(proj.GetAttrValue('AUTHORITY', 1)))
    utm_types.append(proj.GetAttrValue('AUTHORITY', 0))
print(f"UTM Zones:\n {utm_zones}\n")
print(f"UTM Types:\n {utm_types}")

<font face="Calibri" size="3"><b>Identify the most commonly used UTM Zone in the data:</b></font>

In [ ]:
if coord_choice.value == 'UTM':
    utm_unique, counts = np.unique(utm_zones, return_counts=True)
    a = np.where(counts == np.max(counts))
    predominant_utm = utm_unique[a][0]
    print(f"Predominant UTM Zone: {predominant_utm}")
else:
    predominant_utm = '4326'

<font face="Calibri" size="3"><b>Reproject tiffs with errant UTMs to the predominant UTM:</b></font>

In [ ]:
reproject_indicies = [i for i, j in enumerate(utm_zones) if j != predominant_utm] #makes list of indicies in utm_zones that need to be reprojected
print('--------------------------------------------')
print('Reprojecting %4.1f files' %(len(reproject_indicies)))
print('--------------------------------------------')
for k in reproject_indicies:
    temppath = tiff_paths[k].strip()
    _, product_name, tiff_name = temppath.split('/')
    if utm_zones[k] != predominant_utm:
        cmd = f"gdalwarp -overwrite rtc_products/{product_name}/{tiff_name} rtc_products/{product_name}/r{tiff_name} -s_srs {utm_types[k]}:{utm_zones[k]} -t_srs EPSG:{predominant_utm}"
        #print(f"Calling the command: {cmd}")
        !{cmd}
        rm_command = f"rm {tiff_paths[k].strip()}"
        #print(f"Calling the command: {rm_command}")
        !{rm_command}

<font face="Calibri" size="3"><b>Update tiff_paths with any new filenames created during reprojection:</b></font>

In [ ]:
tiff_paths = get_tiff_paths(regex, tiff_pth)
print_tiff_paths(tiff_paths)

<hr>
<font face="Calibri" size="4"> <b>1.3 Merge multiple frames from the same date.</b></font>
<br><br>
<font face="Calibri" size="3"><b>Write a function to collect the product acquisition dates:</b></font>

In [ ]:
def get_dates(product_list):
    dates = []
    for product in product_list:
        dates.append(asfn.date_from_product_name(product).split('T')[0])
    return dates

<font face="Calibri" size="3"><b>Create a list aquisition dates:</b></font>

In [ ]:
dates = get_dates(tiff_paths)
print(dates)

<font face="Calibri" size="3"> You may notice duplicates in your acquisition dates. As HyP3 processes SAR data on a frame-by-frame basis, duplicates may occur if your area of interest is covered by two consecutive  image frames. In this case, two separate images are generated that need to be merged together before time series processing can commence.
<br><br>
<b>Create a set containing each represented date:</b></font>

In [ ]:
unique_dates = set(dates)
print(unique_dates)

<font face="Calibri" size="3"><b>Determine which dates have multiple frames. Create a dictionary with each date as a key linked to a value set as an empty string:</b></font>

In [ ]:
dup_date_batches = [{}]
for date in unique_dates:
    count = 0
    for d in dates:
        if date == d:
            count +=1
    if count > 1:
        dup_date_batches[0].update({date : ""})
if dbl_polar:
    dup_date_batches.append(copy.deepcopy(dup_date_batches[0]))
print(dup_date_batches)

<font face="Calibri" size="3"><b>Update the key values in dup_paths with the string paths to all the tiffs for each date:</b></font>

In [ ]:
if dbl_polar:
    polar_list = [polarization.split(' ')[0], polarization.split(' ')[2]]
else:
    polar_list = [polarization]

for i, polar in enumerate(polar_list):
    polar_regex = f"(\w|/)*_{polar}.(tif|tiff)$"
    polar_paths = get_tiff_paths(polar_regex, tiff_pth)
    for pth in polar_paths:
        date = asfn.date_from_product_name(pth).split('T')[0]
        if date in dup_date_batches[i]:
            dup_date_batches[i][date] = f"{dup_date_batches[i][date]} {pth}"

for d in dup_date_batches:
    print(d)
    print("\n")

<font face="Calibri" size="3"><b>Merge all the frames for each date.</b></font>

In [ ]:
for i, dup_dates in enumerate(dup_date_batches):
    for dup_date in dup_dates:
        output = f"{dup_dates[dup_date].split('/')[0]}/{dup_dates[dup_date].split('/')[1]}/new{dup_dates[dup_date].split('/')[2].split(' ')[0]}"
        gdal_command = f"gdal_merge.py -o {output} {dup_dates[dup_date]}"
        print(f"\n\nCalling the command: {gdal_command}\n")
        !{gdal_command}
        for pth in dup_dates[dup_date].split(' '):
            if pth and asfn.path_exists(pth):
                os.remove(pth)
                print(f"Deleting: {pth}")

<hr>
<font face="Calibri" size="3"> <b>Verify that all duplicate dates were resolved:</b> </font>

In [ ]:
tiff_paths = get_tiff_paths(regex, tiff_pth)
for polar in polar_list:
    polar_tiff_pth = tiff_pth.replace('V*', polar)
    polar_tiff_paths = get_tiff_paths(regex, polar_tiff_pth)
    dates = get_dates(polar_tiff_paths)
    if len(dates) != len(set(dates)):
        print(f"Duplicate dates still present!")
    else:
        print(f"No duplicate dates are associated with {polar} polarization.")

<font face="Calibri" size="3"><b>Print the updated tif paths:</b></font>

In [ ]:
print_tiff_paths(tiff_paths)

<hr>
<font face="Calibri">

<font size="5"> <b> 2. Create Subset and Stack Up Your Data </b> </font>

<font size="3"> Now you are ready to work with your data. The next cells allow you to select an area of interest (AOI; via bounding-box corner coordinates) for your data analysis. Once selected, the AOI is being extracted and a data stack is formed.

<b>Create a string containing paths to one image for each area represented in the stack:</b>
</font>
</font>

In [ ]:
to_merge = {}
for pth in tiff_paths:
    info = gdal.Info(pth, format='json')['wgs84Extent']['coordinates']

    coords = [info[0][0], info[0][3]]
    for i in range(0, 2):
        for j in range(0, 2):
            coords[i][j] = round(coords[i][j])
    str_coords = f"{str(coords[0])}{str(coords[1])}"
    if str_coords not in to_merge:
        to_merge.update({str_coords: pth})
print(to_merge)
print()
merge_paths = ""
for pth in to_merge:
    merge_paths = f"{merge_paths} {to_merge[pth]}"

print(merge_paths)

<font face="Calibri" size="3"><b>Merge the images, creating a full scene for display in the Area-Of-Interest selector:</b></font>

In [ ]:
full_scene = f"{analysis_directory}/full_scene.tif"
if os.path.exists(full_scene):
    os.remove(full_scene)
gdal_command = f"gdal_merge.py -o {full_scene} {merge_paths}"
!{gdal_command}

<font face="Calibri" size="3"><b>Create a VRT of the full scene:</b></font>

In [ ]:
image_file = f"{analysis_directory}/raster_stack.vrt"
!gdalbuildvrt -separate $image_file -overwrite $full_scene

<font face="Calibri" size="3"><b>Convert the VRT into an array:</b> </font>

In [ ]:
img = gdal.Open(image_file)
rasterstack = img.ReadAsArray()

<font face="Calibri" size="3"><b>Print the number of bands, pixels, and lines:</b> </font>

In [ ]:
print(img.RasterCount) # Number of Bands
print(img.RasterXSize) # Number of Pixels
print(img.RasterYSize) # Number of Lines

<font face="Calibri" size="3"><b>Create an AOI selector from an image in your raster stack:</b> </font>

In [ ]:
%matplotlib notebook
fig_xsize = 7.5
fig_ysize = 7.5
aoi = asfn.AOI_Selector(rasterstack, fig_xsize, fig_ysize)

<font face="Calibri" size="3"><b>Gather and define projection details:</b> </font>

In [ ]:
geotrans = img.GetGeoTransform()
projlatlon = pyproj.Proj('EPSG:4326') # WGS84
projimg = pyproj.Proj(f'EPSG:{predominant_utm}')

<font face="Calibri" size="3"><b>Write a function to convert the pixel, line coordinates from the AOI selector into geographic coordinates in the stack's EPSG projection:</b> </font>

In [ ]:
def geolocation(x, y, geotrans,latlon=True):
    ref_x = geotrans[0]+x*geotrans[1]
    ref_y = geotrans[3]+y*geotrans[5]
    if latlon:
        ref_y, ref_x = pyproj.transform(projimg, projlatlon, ref_x, ref_y)
    return [ref_x, ref_y]

<font face="Calibri" size="3"><b>Call geolocation to gather the aoi_coords:</b> </font>

In [ ]:
try:
    aoi_coords = [geolocation(aoi.x1, aoi.y1, geotrans, latlon=False), geolocation(aoi.x2, aoi.y2, geotrans, latlon=False)]
    print(f"aoi_coords in EPSG {predominant_utm}: {aoi_coords}")
except TypeError:
    print('TypeError')
    display(Markdown(f'<text style=color:red>This error occurs if an AOI was not selected.</text>'))
    display(Markdown(f'<text style=color:red>Note that the square tool icon in the AOI selector menu is <b>NOT</b> the selection tool. It is the zoom tool.</text>'))
    display(Markdown(f'<text style=color:red>Read the tips above the AOI selector carefully.</text>'))

<font face="Calibri" size="3"><b>Collect the paths to the tiffs:</b> </font>

In [ ]:
tiff_paths = get_tiff_paths(regex, polarization, tiff_pth)
#print_tiff_paths(tiff_paths)

<font face="Calibri" size="3"><b>Create a subdirectory in which to store the subset tiffs:</b> </font>

In [ ]:
print("Choose a directory name in which to store the subset geotiffs.")
print("Note: this will sit alongside the directory containing your pre-subset geotiffs.")
while True:
    sub_name = input()
    if sub_name == "":
        print("Please enter a valid directory name")
        continue
    else:
        break

<font size="3"><b>Subset the tiffs and move them from the individual product directories into their own directory, /tiffs:</b></font>

In [ ]:
subset_dir = f"{analysis_directory}/{sub_name}/"
asfn.new_directory(subset_dir)
for i, tiff_path in enumerate(tiff_paths):
    for name_chunk in tiff_path.split('/')[-1].split('_'):
        nums = list(range(48, 58))
        if len(name_chunk) == 15 and ord(name_chunk[0]) in nums:
            date = name_chunk.split('T')[0]
            break
        elif len(name_chunk) == 8 and ord(name_chunk[0]) in nums:
                date = name_chunk
                break
    polar = tiff_path.split('/')[-1].split('.')[0][-2:]
    print(f"\nProduct #{i+1}:")
    gdal_command = f"gdal_translate -projwin {aoi_coords[0][0]} {aoi_coords[0][1]} {aoi_coords[1][0]} {aoi_coords[1][1]} -projwin_srs 'EPSG:{predominant_utm}' -co \"COMPRESS=DEFLATE\" -a_nodata 0 {tiff_path} {subset_dir}{date}_{polar}.tiff"
    print(f"Calling the command: {gdal_command}")
    !{gdal_command}

<font size="3"><b>Grab the updated paths of the images:</b></font>

In [ ]:
subset_pth = f"{sub_name}/*.tif*"
tiff_paths = get_tiff_paths(regex, subset_pth)
print_tiff_paths(tiff_paths)

<font size="3"><b>Delete any subset tifs that are filled with NaNs and contain no data.</b></font>

In [ ]:
asfn.remove_nan_filled_tifs(subset_dir, tiff_paths)
tiff_paths = get_tiff_paths(regex, subset_pth)
print(f"\nThere are {len(tiff_paths)} tiffs remaining in the image stack.")

<font size="3"><b>Update the list of dates and tiff_paths after removing NaN filled images:</b></font>

In [ ]:
dates = []
pth = glob.glob(f"{subset_dir}/*.tif*")
pth.sort()
for p in pth:
    date = os.path.basename(p)[0:8]
    dates.append(date)
    print(date)


# print_tiff_paths(tiff_paths) # uncomment to print tiff paths

<hr>
<font face="Calibri" size="3"> Now we stack up the data by creating a virtual raster table with links to all subset data files: </font>
<br><br>
<font size="3"><b>Create the virtual raster table for the subset GeoTiffs:</b></font>

In [ ]:
!gdalbuildvrt -separate raster_stack.vrt $subset_dir/*.tif*

<hr>
<font face="Calibri">

<font size="5"> <b> 3. Now You Can Work With Your Data </b> </font>

<font size="3"> Now you are ready to perform time series change detection on your data stack.
</font>
</font>

<br>
<font face="Calibri" size="4"> <b> 3.1 Define Data Directory and Path to VRT </b> </font>
<br><br>
<font face="Calibri" size="3"><b>Create a variable containing the VRT filename:</b></font>

In [ ]:
image_file = "raster_stack.vrt"

<font face="Calibri" size="3"><b>Create an index of timedelta64 data with Pandas:</b></font>

In [ ]:
# Get some indices for plotting
time_index = pd.DatetimeIndex(dates)

<font face="Calibri" size="3"><b>Print the bands and dates for all images in the virtual raster table (VRT):</b></font>

In [ ]:
j = 1
print(f"Bands and dates for {image_file}")
for i in time_index:
    print("{:4d} {}".format(j, i.date()), end=' ')
    j += 1
    if j%5 == 1:
        print()

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.2 Open Your Data Stack and Visualize Some Layers </b> </font>

<font face="Calibri" size="3"> We will <b>open your VRT</b> and visualize some layers using Matplotlib. </font>

In [ ]:
img = gdal.Open(image_file)

<font face="Calibri" size="3"><b>Print the bands, pixels, and lines:</b></font>

In [ ]:
print(f"Number of  bands: {img.RasterCount}")
print(f"Number of pixels: {img.RasterXSize}")
print(f"Number of  lines: {img.RasterYSize}")

<font face="Calibri" size="3"><b>Read in raster data for the first two bands:</b></font>

In [ ]:
# change the number passed to GetRasterBand() to read rasters from different bands
# Note: you must pass a valid band number to GetRasterBand()
raster_1 = np.ma.masked_invalid(img.GetRasterBand(1).ReadAsArray()).filled(0)
raster_2 = np.ma.masked_invalid(img.GetRasterBand(2).ReadAsArray()).filled(0)

<font face="Calibri" size="3"><b>Plot images and histograms for bands 1 and 2:</b>
<br>
Note: Depending on the histograms plotted by this cell, you may wish to adjust vmax when calling imshow() on ax1 and ax3. Increase the vmax value if the histogram cuts off much of the end of the peak, making your image too bright to see features well. Decrease vmax if the histogram extends much beyond the end of the peak, which will make your image appear dark. </font>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plb
# Setup the pyplot plots
fig = plb.figure(figsize=(18,10)) # Initialize figure with a size
ax1 = fig.add_subplot(221)  # 221 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(222)  # 222 determines: 2 rows, 2 plots, second plot
ax3 = fig.add_subplot(223)  # 223 determines: 2 rows, 2 plots, third plot
ax4 = fig.add_subplot(224)  # 224 determines: 2 rows, 2 plots, fourth plot

# Plot the band 1 image
band_number = 1
ax1.imshow(raster_1,cmap='gray', vmin=0, vmax=0.3) # see note above regarding vmax adjustments
ax1.set_title('Image Band {} {}'.format(band_number, time_index[band_number-1].date()))

# Flatten the band 1 image into a 1 dimensional vector and plot the histogram:
h = ax2.hist(raster_1.flatten(), bins=200, range=(0, 0.3))
ax2.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax2.set_title('Histogram Band {} {}'.format(band_number, time_index[band_number-1].date()))

# Plot the band 2 image
band_number = 2
ax3.imshow(raster_2,cmap='gray', vmin=0, vmax=0.3) # see note above regarding vmax adjustments
ax3.set_title('Image Band {} {}'.format(band_number, time_index[band_number-1].date()))

# Flatten the band 2 image into a 1 dimensional vector and plot the histogram:
h = ax4.hist(raster_2.flatten(),bins=200,range=(0,0.3))
ax4.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax4.set_title('Histogram Band {} {}'.format(band_number, time_index[band_number-1].date()))

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.3 Create a Time Series Animation </b> </font>

<font face="Calibri" size="3"><b>Create a directory in which to store our plots and animations:</b></font>

In [ ]:
output_path = 'plots_animations'
asfn.new_directory(output_path)

<font face="Calibri" size="3"> Now we are ready to <b>create a time series animation</b> from the calibrated SAR data. </font>

In [ ]:
raster0 = img.GetRasterBand(1).ReadAsArray()
band_number = 0 # Needed for updates
raster_stack = img.ReadAsArray()
del img

<font face="Calibri" size="3"><b>Create a masked raster stack:</b></font>

In [ ]:
raster_stack_masked = np.ma.masked_where(raster_stack==0, raster_stack)

<font face="Calibri" size="3"><b>Generate a matplotlib time-series animation:</b></font>

In [ ]:
%%capture
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111)
ax.axis('off')
vmin = np.percentile(raster_stack.flatten(), 5)
vmax = np.percentile(raster_stack.flatten(), 95)

r0dB = 20 * np.log10(raster0) - 83

im = ax.imshow(raster0, cmap='gray', vmin=vmin, vmax=vmax)
ax.set_title("{}".format(time_index[0].date()))

def animate(i):
    ax.set_title("{}".format(time_index[i].date()))
    im.set_data(raster_stack[i])

# Interval is given in milliseconds
ani = animation.FuncAnimation(fig, animate, frames=raster_stack.shape[0], interval=400)

<font face="Calibri" size="3"><b>Configure matplotlib's RC settings for the animation:</b></font>

In [ ]:
rc('animation', embed_limit=40971520.0)

<font face="Calibri" size="3"><b>Create a javascript animation of the time-series running inline in the notebook:</b></font>

In [ ]:
HTML(ani.to_jshtml())

<font face="Calibri" size="3"><b>Delete the dummy png</b> that was saved to the current working directory while generating the javascript animation in the last code cell.</font>

In [ ]:
try:
    os.remove('None0000000.png')
except FileNotFoundError:
    pass

<font face="Calibri" size="3"><b>Save the animation as a gif:</b> </font>

In [ ]:
ani.save(f"{output_path}/animation.gif", writer='pillow', fps=2)

<br>
<hr>
<font face="Calibri" size="5"> <b> 4. Cumulative Sum-based Change Detection Across an Entire Image</b> </font>

<font face="Calibri" size="3"> Using numpy arrays we can apply the concept of **cumulative sum change detection** analysis effectively on the entire image stack. We take advantage of array slicing and axis-based computing in numpy. **Axis 0 is the time domain** in our raster stacks.

<hr>
<font size="4"><b>4.1 Create our time series stack</b></font>
<br><br>
<font size="3"><b>Calculate the dB scale:</b></font>

In [ ]:
db = 10.*np.ma.log10(raster_stack_masked)

<font face="Calibri" size="3">Sometimes it makes sense to <b>extract a reduced time span</b> from the full time series to reduce the number of different change objects in a scene. In the following, we extract a shorter time span:
</font>

In [ ]:
date_picker = asfn.gui_date_picker(dates)
date_picker

In [ ]:
subset_dates = date_picker.value
subset_dates = pd.DatetimeIndex(subset_dates)
date_index_subset = np.where((time_index>=subset_dates[0]) & (time_index<=subset_dates[1]))
db_subset = np.squeeze(db[date_index_subset, :, :])
time_index_subset = time_index[date_index_subset]

In [ ]:
plt.figure(figsize=(12, 8))
band_number = 0
vmin = np.percentile(db_subset[band_number], 5)
vmax = np.percentile(db_subset[band_number], 95)
plt.title(f'Band  {band_number+1} {time_index_subset[band_number].date()}')
plt.imshow(db_subset[0], cmap='gray', vmin=vmin, vmax=vmax)
cbar = plt.colorbar()
_ = cbar.ax.set_xlabel('dB', fontsize='12')

<br>
<hr>
<font face="Calibri" size="4"> <b> 4.2 Calculate Mean Across Time Series to Prepare for Calculation of Cummulative Sum $S$:</b> </font>
<br><br>
<font face="Calibri" size="3"><b>Write a function to convert our plots into GeoTiffs:</b></font>

In [ ]:
def geotiff_from_plot(source_image, out_filename, extent, cmap=None, vmin=None, vmax=None, interpolation=None, dpi=300):
    assert "." not in out_filename, 'Error: Do not include the file extension in out_filename'
    assert type(extent) == list and len(extent) == 2 and len(extent[0]) == 2 and len(
        extent[1]) == 2, 'Error: extent must be a list in the form [[upper_left_x, upper_left_y], [lower_right_x, lower_right_y]]'

    plt.figure()
    plt.axis('off')
    plt.imshow(source_image, cmap=cmap, vmin=vmin, vmax=vmax, interpolation=interpolation)
    temp = f"{out_filename}_temp.png"
    plt.savefig(temp, dpi=dpi, transparent='true', bbox_inches='tight', pad_inches=0)

    cmd = f"gdal_translate -of Gtiff -a_ullr {extent[0][0]} {extent[0][1]} {extent[1][0]} {extent[1][1]} -a_srs EPSG:{predominant_utm} {temp} {out_filename}.tiff"
    !{cmd}
    try:
        os.remove(temp)
    except FileNotFoundError:
        pass

<font face="Calibri" size="3"><b>Plot the time-series mean and save as a png (time_series_mean.png):</b></font>

In [ ]:
db_mean = np.mean(db_subset, axis=0)
plt.figure(figsize=(12, 8))
plt.imshow(db_mean, cmap='gray')
cbar = plt.colorbar()
cbar.ax.set_xlabel('dB', fontsize='12')
plt.savefig(f"{output_path}/time_series_mean.png", dpi=300, transparent='true')

<font face="Calibri" size="3"><b>Save the time-series mean as a GeoTiff (time_series_mean.tiff):</b></font>

In [ ]:
%%capture
geotiff_from_plot(db_mean, f"{output_path}/time_series_mean", aoi_coords, cmap='gray')

<font face="Calibri" size="3"><b>Calculate the residuals and plot residuals[0]. Save it as a png (residuals.png):</b></font>

In [ ]:
residuals = db_subset - db_mean

plt.figure(figsize=(12, 8))
plt.imshow(residuals[0])
plt.title('Residuals for Band  {} {}'.format(band_number+1, time_index_subset[band_number].date()))
cbar = plt.colorbar()
_ = cbar.ax.set_xlabel('dB', fontsize='12')
plt.savefig(f"{output_path}/residuals.png", dpi=300, transparent='true')

<font face="Calibri" size="3"><b>Save the residuals[0] as a GeoTiff (residuals.tiff):</b></font>

In [ ]:
%%capture
geotiff_from_plot(residuals[0], f"{output_path}/residuals", aoi_coords)

<br>
<hr>
<font face="Calibri" size="4"><b> 4.3 Calculate Cummulative Sum $S$ as well as Change Magnitude $S_{diff}$:</b></font>
<br><br>
<font face="Calibri" size="3"><b>Plot Smin, Smax, and the change magnitude and save a png of the plots (Smin_Smax_Sdiff.png):</b></font>

In [ ]:
summation = np.cumsum(residuals, axis=0)
summation_max = np.max(summation, axis=0)
summation_min = np.min(summation, axis=0)
change_mag = summation_max - summation_min
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
vmin = np.percentile(summation_min.flatten(), 3)
vmax = np.percentile(summation_max.flatten(), 97)
max_plot = ax[0].imshow(summation_max, vmin=vmin, vmax=vmax)
ax[0].set_title('$S_{max}$')
ax[1].imshow(summation_min, vmin=vmin, vmax=vmax)
ax[1].set_title('$S_{min}$')
ax[2].imshow(change_mag, vmin=vmin, vmax=vmax)
ax[2].set_title('Change Magnitude')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(max_plot, cax=cbar_ax)
_ = cbar.ax.set_xlabel('dB', fontsize='12')
plt.savefig(f"{output_path}/Smin_Smax_Sdiff.png", dpi=300, transparent='true')

<font face="Calibri" size="3"><b>Save Smax as a GeoTiff (Smax.tiff):</b></font>

In [ ]:
%%capture
geotiff_from_plot(summation_max, f"{output_path}/Smax", aoi_coords, vmin=vmin, vmax=vmax)

<font face="Calibri" size="3"><b>Save Smin as a GeoTiff (Smin.tiff):</b></font>

In [ ]:
%%capture
geotiff_from_plot(summation_min, f"{output_path}/Smin", aoi_coords, vmin=vmin, vmax=vmax)

<font face="Calibri" size="3"><b>Save the change magnitude as a GeoTiff (Sdiff.tiff):</b></font>

In [ ]:
%%capture
geotiff_from_plot(change_mag, f"{output_path}/Sdiff", aoi_coords, vmin=vmin, vmax=vmax)

<br>
<hr>
<font face="Calibri" size="4"> <b> 4.4 Mask $S_{diff}$ With a-priori Threshold To Idenfity Change Candidates:</b> </font>

<font face="Calibri" size="3">To identify change candidate pixels, we can threshold $S_{diff}$ to reduce computation of the bootstrapping. For land cover change, we would not expect more than 5-10% change pixels in a landscape. So, if the test region is reasonably large, setting a threshold for expected change to 10% is appropriate. In our example, we'll start out with a very conservative threshold of 50%.
<br><br>
<b>Plot and tsave the histogram and CDF for the change magnitude (change_mag_histogram_CDF.png):</b></font>

In [ ]:
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(14, 6)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(122)
# Second plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten*
# the two-dimensional image into a one-dimensional vector.
histogram = ax1.hist(change_mag.flatten(), bins=200, range=(0, np.max(change_mag)))
ax1.xaxis.set_label_text('Change Magnitude')
ax1.set_title('Change Magnitude Histogram')
plt.grid()
n, bins, patches = ax2.hist(change_mag.flatten(), bins=200, range=(0, np.max(change_mag)), cumulative='True', density='True', histtype='step', label='Empirical')
ax2.xaxis.set_label_text('Change Magnitude')
ax2.set_title('Change Magnitude CDF')
plt.grid()
plt.savefig(f"{output_path}/change_mag_histogram_CDF", dpi=72)

In [ ]:
precentile = 0.5
out_indicies = np.where(n>precentile)
threshold_index = np.min(out_indicies)
threshold = bins[threshold_index]
print('At the {}% percentile, the threshold value is {:2.2f}'.format(precentile*100, threshold))

<font face="Calibri" size="3">Using this threshold, we can <b>visualize our change candidate areas and save them as a png (change_candidate.png):</b></font>

In [ ]:
change_mag_mask = change_mag < threshold
plt.figure(figsize=(12, 8))
plt.title('Change Candidate Areas (black)')
_ = plt.imshow(change_mag_mask, cmap='gray')
plt.savefig(f"{output_path}/change_candidate.png", dpi=300, transparent='true')

<font face="Calibri" size="3"><b>Save the change candidate areas as a GeoTiff (change_canididate.tiff):</b>
</font>

In [ ]:
%%capture
geotiff_from_plot(change_mag_mask, f"{output_path}/change_candidate", aoi_coords, cmap='gray')

<br>
<hr>
<font face="Calibri" size="4"> <b> 4.5 Bootstrapping to Prepare for Change Point Selection:</b> </font>

<font face="Calibri" size="3">We can now perform bootstrapping over the candidate pixels. The workflow is as follows:
<ul>
    <li>Filter our residuals to the change candidate pixels</li>
    <li>Perform bootstrapping over candidate pixels</li>
</ul>
For efficient computing we permutate the index of the time axis.
</font>

In [ ]:
residuals_mask = np.broadcast_to(change_mag_mask , residuals.shape)
residuals_masked = np.ma.array(residuals, mask=residuals_mask)

<font face="Calibri" size="3">On the masked time series stack of residuals, we can re-compute the cumulative sums:
</font>

In [ ]:
summation_masked = np.ma.cumsum(residuals_masked, axis=0)

<font face="Calibri" size="3"><b>Plot the masked Smax, Smin, and change magnitude. Save them as a png (masked_Smax_Smin_Sdiff.png):</b>
</font>

In [ ]:
summation_masked_max = np.ma.max(summation_masked, axis=0)
summation_masked_min = np.ma.min(summation_masked, axis=0)
change_mag_masked = summation_masked_max - summation_masked_min
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
vmin = summation_masked_min.min()
vmax = summation_masked_max.max()
masked_sum_max_plot = ax[0].imshow(summation_masked_max, vmin=vmin, vmax=vmax)
ax[0].set_title('Masked $S_{max}$')
ax[1].imshow(summation_masked_min, vmin=vmin, vmax=vmax)
ax[1].set_title('Masked $S_{min}$')
ax[2].imshow(change_mag_masked, vmin=vmin, vmax=vmax)
ax[2].set_title('Masked Change Magnitude')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(masked_sum_max_plot, cax=cbar_ax)
_ = cbar.ax.set_xlabel('dB', fontsize='12')
plt.savefig(f"{output_path}/masked_Smax_Smin_Sdiff.png", dpi=300, transparent='true')

<font face="Calibri" size="3"><b>Save the masked Smax as a GeoTiff (masked_Smax.tiff):</b>
</font>

In [ ]:
%%capture
geotiff_from_plot(summation_masked_max, f"{output_path}/masked_Smax", aoi_coords, vmin=vmin, vmax=vmax)

<font face="Calibri" size="3"><b>Save the masked Smin as a GeoTiff (masked_Smin.tiff):</b>
</font>

In [ ]:
%%capture
geotiff_from_plot(summation_masked_min, f"{output_path}/masked_Smin", aoi_coords, vmin=vmin, vmax=vmax)

<font face="Calibri" size="3"><b>Save the masked change magnitude as a GeoTiff (masked_Sdiff.tiff):</b>
</font>

In [ ]:
%%capture
geotiff_from_plot(change_mag_masked, f"{output_path}/masked_Sdiff", aoi_coords, vmin=vmin, vmax=vmax)

<font face="Calibri" size="3">Now let's perform <b>bootstrapping</b>:
</font>

In [ ]:
random_index = np.random.permutation(residuals_masked.shape[0])
residuals_random = residuals_masked[random_index,:,:]

In [ ]:
n_bootstraps = 100  # bootstrap sample size

# to keep track of the maximum Sdiff of the bootstrapped sample:
change_mag_random_max = np.ma.copy(change_mag_masked)
change_mag_random_max[~change_mag_random_max.mask]=0
# to compute the Sdiff sums of the bootstrapped sample:
change_mag_random_sum = np.ma.copy(change_mag_masked)
change_mag_random_sum[~change_mag_random_max.mask]=0
# to keep track of the count of the bootstrapped sample
n_change_mag_gt_change_mag_random = np.ma.copy(change_mag_masked)
n_change_mag_gt_change_mag_random[~n_change_mag_gt_change_mag_random.mask]=0
print("Running Bootstrapping for %4.1f iterations ..." % (n_bootstraps))
for i in range(n_bootstraps):
    # For efficiency, we shuffle the time axis index and use that
    #to randomize the masked array
    random_index = np.random.permutation(residuals_masked.shape[0])
    # Randomize the time step of the residuals
    residuals_random = residuals_masked[random_index,:,:]
    summation_random = np.ma.cumsum(residuals_random, axis=0)
    summation_random_max = np.ma.max(summation_random, axis=0)
    summation_random_min = np.ma.min(summation_random, axis=0)
    change_mag_random = summation_random_max - summation_random_min
    change_mag_random_sum += change_mag_random
    change_mag_random_max[np.ma.greater(change_mag_random, change_mag_random_max)] = \
    change_mag_random[np.ma.greater(change_mag_random, change_mag_random_max)]
    n_change_mag_gt_change_mag_random[np.ma.greater(change_mag_masked, change_mag_random)] += 1
    if ((i+1)/n_bootstraps*100)%10 == 0:
        print("\r%4.1f%% completed" % ((i+1)/n_bootstraps*100), end='\r', flush=True)
print(f"Bootstrapping Complete")

<br>
<hr>
<font face="Calibri" size="4"> <b> 4.6 Extract Confidence Metrics and Select Final Change Points:</b> </font>

<font face="Calibri" size="3">We first <b>compute for all pixels the confidence level $CL$, the change point significance metric $CP_{significance}$ and the product of the two as our confidence metric for identified change points. Plot the results and save them as a png (confidenceLevel_CPSignificance.png):</b></font>

In [ ]:
confidence_level = n_change_mag_gt_change_mag_random / n_bootstraps
change_point_significance = 1.- (change_mag_random_sum / n_bootstraps)/change_mag
#Plot
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
a = ax[0].imshow(confidence_level*100)
cbar0 = fig.colorbar(a, ax=ax[0])
_ = cbar0.ax.set_xlabel('%', fontsize='12')
ax[0].set_title('Confidence Level %')
a = ax[1].imshow(change_point_significance)
_ = fig.colorbar(a, ax=ax[1])
ax[1].set_title('Significance')
a = ax[2].imshow(confidence_level*change_point_significance)
_ = fig.colorbar(a, ax=ax[2])
_ = ax[2].set_title('CL x S')
plt.savefig(f"{output_path}/confidenceLevel_CPSignificance.png", dpi=300, transparent='true')

<font face="Calibri" size="3"><b>Save the confidence level as a GeoTiff (confidence_level.tiff):</b>
</font>

In [ ]:
%%capture
geotiff_from_plot(confidence_level*100, f"{output_path}/confidence_level", aoi_coords)

<font face="Calibri" size="3"><b>Save the change point significance as a GeoTiff (cp_significance.tiff):</b>
</font>

In [ ]:
%%capture
geotiff_from_plot(change_point_significance, f"{output_path}/cp_significance", aoi_coords)

<font face="Calibri" size="3"><b>Save the change point significance as a GeoTiff (cp_significance.tiff):</b>
</font>

In [ ]:
%%capture
geotiff_from_plot(confidence_level*change_point_significance, f"{output_path}/confidenceLevel_x_CPSignificance", aoi_coords)

<font face="Calibri" size="3">Now we can <b>set a change point threshold</b> to identify most likely change pixels in our map of change candidates:
</font>

In [ ]:
change_point_threshold = 0.01

<font face="Calibri" size="3"><b>Plot the detected change pixels based on the change_point_threshold and save it as a png (detected_change_pixels.png):</b></font>

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1)
plt.title('Detected Change Pixels based on Threshold %2.2f' % (change_point_threshold))
a = ax.imshow(confidence_level*change_point_significance < change_point_threshold, cmap='cool')
plt.savefig(f"{output_path}/detected_change_pixels.png", dpi=300, transparent='true')

<font face="Calibri" size="3"><b>Save the detected_change_pixels as a GeoTiff (detected_change_pixels.tiff):</b>
</font>

In [ ]:
%%capture
geotiff_from_plot(confidence_level*change_point_significance < change_point_threshold, f"{output_path}/detected_change_pixels", aoi_coords, cmap='cool')

<br>
<hr>
<font face="Calibri" size="4"> <b> 4.7 Derive Timing of Change for Each Change Pixel:</b> </font>

<font face="Calibri" size="3">Our last step in the identification of the change points is to extract the timing of the change. We will produce a raster layer that shows the band number of this first date after a change was detected. We will make use of the numpy indexing scheme. First, we create a combined mask of the first threshold and the identified change points after the bootstrapping. For this we use the numpy "mask_or" operation.
</font>

In [ ]:
# make a mask of our change points from the new threhold and the previous mask
change_point_mask = np.ma.mask_or(confidence_level*change_point_significance < change_point_threshold, confidence_level.mask)
# Broadcast the mask to the shape of the masked S curves
change_point_mask2 = np.broadcast_to(change_point_mask, summation_masked.shape)
# Make a numpy masked array with this mask
change_point_raster = np.ma.array(summation_masked.data, mask=change_point_mask2)

<font face="Calibri" size="3">To retrieve the dates of the change points we find the band indices in the time series along the time axis where the maximum of the cumulative sums was located. Numpy offers the "argmax" function for this purpose.
</font>

In [ ]:
change_point_index = np.ma.argmax(change_point_raster, axis=0)
change_indices = list(np.unique(change_point_index))
change_indices.remove(0)
print(change_indices)
# Look up the dates from the indices to get the change dates
all_dates = time_index_subset
change_dates = [str(all_dates[x].date()) for x in change_indices]

<font face="Calibri" size="3">Lastly, we <b>plot the change dates by showing the $CP_{index}$ raster and label the change dates. Save the plot as a png (change_dates.png):</b></font>

In [ ]:
ticks = change_indices
ticklabels = change_dates

cmap = plt.cm.get_cmap('tab20', ticks[-1])
fig, ax = plt.subplots(figsize=(12, 24))
cax = ax.imshow(change_point_index, interpolation='nearest', cmap=cmap)
# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(p,cax=cbar_ax)

ax.set_title('Dates of Change')
# cbar = fig.colorbar(cax,ticks=ticks)
cbar = fig.colorbar(cax, ticks=ticks, orientation='horizontal')
_ = cbar.ax.set_xticklabels(ticklabels, size=10, rotation=45, ha='right')
plt.savefig(f"{output_path}/change_dates.png", dpi=300, transparent='true')

<font face="Calibri" size="3"><b>Save the change dates as a GeoTiff (change_dates.tiff):</b>
</font>

In [ ]:
%%capture
geotiff_from_plot(change_point_index, f"{output_path}/change_dates", aoi_coords, cmap=cmap, interpolation='nearest', dpi=600)

<font face="Calibri" size="2"> <i>Change_Detection_Amplitude_Time_Series_HyP3_v2.ipynb - Version 1.0 - February 2021 </i></font>